# How to access to Iris data? 

In this example, we will show how to access to Iris data in Python via SQL queries.  
In particular, we will select the reply data of the last measurement of the Iris's exhaustive data series.

In [43]:
import os
from iris_client import IrisClient, AsyncIrisClient


# Iris credentials
base_url = "https://api.iris.dioptra.io"
username = os.environ.get("IRIS_USERNAME")
password = os.environ.get("IRIS_PASSWORD")
if not username or not password:
    raise ValueError("IRIS_USERNAME and IRIS_PASSWORD environment variables must be set")

# Check out https://github.com/dioptra-io/iris-client for more info on IrisClient
with IrisClient(base_url, username, password) as iris:
    measurements = iris.all("/measurements/public", params={"tag": "collection:exhaustive"})

In [44]:
# We select the measurement we want to query (newest measurement)
measurement = measurements[0]

In [45]:
# Get temporary access token to the database for this measurement
with IrisClient(base_url, username, password) as iris:
    me = iris.get("/users/me/services", params={"measurement_uuid": measurement["uuid"]}).json()
    credentials = dict(
        base_url=me["clickhouse"]["base_url"],
        database="iris",
        username=me["clickhouse"]["username"],
        password=me["clickhouse"]["password"]
    )

In [46]:
def format_uuid(uuid):
    return uuid.replace("-", "_")

def get_tables(measurement, data_type="results"):
    """
    Get the table of each agent in the measurement.
    """
    return [
        f"{data_type}__{format_uuid(measurement['uuid'])}__{format_uuid(agent['agent_uuid'])}" 
        for agent in measurement["agents"]
    ]

In [47]:
# Get reply table names for this measurement
tables = get_tables(measurement)

In [48]:
from pych_client import ClickHouseClient

# Check out https://github.com/dioptra-io/pych-client for more info on ClickHouseClient
with ClickHouseClient(**credentials) as client:
    for row in client.iter_json(
        f"SELECT probe_src_addr, probe_dst_prefix, probe_ttl, reply_src_addr FROM {tables[0]} LIMIT 10"
    ):
        print(row)

{'probe_src_addr': '::ffff:132.227.123.8', 'probe_dst_prefix': '::ffff:1.0.0.0', 'probe_ttl': 8, 'reply_src_addr': '::ffff:62.40.98.178'}
{'probe_src_addr': '::ffff:132.227.123.8', 'probe_dst_prefix': '::ffff:1.0.0.0', 'probe_ttl': 11, 'reply_src_addr': '::ffff:141.101.71.2'}
{'probe_src_addr': '::ffff:132.227.123.8', 'probe_dst_prefix': '::ffff:1.0.0.0', 'probe_ttl': 12, 'reply_src_addr': '::ffff:1.0.0.13'}
{'probe_src_addr': '::ffff:132.227.123.8', 'probe_dst_prefix': '::ffff:1.0.0.0', 'probe_ttl': 13, 'reply_src_addr': '::ffff:1.0.0.13'}
{'probe_src_addr': '::ffff:132.227.123.8', 'probe_dst_prefix': '::ffff:1.0.0.0', 'probe_ttl': 14, 'reply_src_addr': '::ffff:1.0.0.13'}
{'probe_src_addr': '::ffff:132.227.123.8', 'probe_dst_prefix': '::ffff:1.0.0.0', 'probe_ttl': 15, 'reply_src_addr': '::ffff:1.0.0.13'}
{'probe_src_addr': '::ffff:132.227.123.8', 'probe_dst_prefix': '::ffff:1.0.0.0', 'probe_ttl': 16, 'reply_src_addr': '::ffff:1.0.0.13'}
{'probe_src_addr': '::ffff:132.227.123.8', 'prob